In [11]:
import json
from pprint import pprint
import secrets as secrets_module

from dydx3 import Client
from dydx3 import private_key_to_public_key_pair_hex
from dydx3.constants import NETWORK_ID_ROPSTEN, NETWORK_ID_MAINNET
from dydx3.constants import API_HOST_ROPSTEN, API_HOST_MAINNET
from web3 import Web3

def generate_private_key():
    priv = secrets_module.token_hex(32)
    private_key = "0x" + priv
    return private_key

def onboard_initial_account(eth_private_key, infura_endpoint, network_id, host):   
    w3 = Web3(Web3.HTTPProvider(infura_endpoint))
    eth_address = w3.eth.account.privateKeyToAccount(eth_private_key).address
    print(f"Eth Address: {eth_address}\n")

    dydx_client = Client(
        network_id=network_id,
        host=host,
        eth_private_key=eth_private_key,
        web3=w3,
    )
    
    stark_private_key = dydx_client.onboarding.derive_stark_key()
    dydx_client.stark_private_key = stark_private_key
    print(f"STARK private key: {stark_private_key}\n")
    
    public_x, public_y = private_key_to_public_key_pair_hex(stark_private_key)
    print(f"STARK (public x, public y): ({public_x},{public_y})\n")
    
    onboarding_response = dydx_client.onboarding.create_user(
        stark_public_key=public_x,
        stark_public_key_y_coordinate=public_y,
    ).data
    print("Account onboarding response:")
    pprint(onboarding_response)
    return

secrets = {}
# ensure that the secrests.json file contains `infura_ropsten_endpoint` attribute
with open("secrets.json") as secretfile:
    secrets = json.load(secretfile)

# ropsten check (with randomly generated ethereum address)
onboard_initial_account(
    generate_private_key(),
    secrets["infura_ropsten_endpoint"],
    NETWORK_ID_ROPSTEN,
    API_HOST_ROPSTEN
)

# # mainnet check
# onboard_initial_account(
#     secrets["eth_private_endpoint"], # must provide a used account's private key, else get `DydxApiError(status_code=400, response={'errors': [{'msg': 'User wallet has no transactions, Ethereum or USDC'}]})`
#     secrets["infura_mainnet_key"],
#     NETWORK_ID_MAINNET,
#     API_HOST_MAINNET
# )

Eth Address: 0xF0E650cf8f99Bc8b69FDa2178CEFFB2B90DFd34A

STARK private key: 0x3532166e51abd644ca60da862b1908a95b4461fe1f155c568e75b6e28dc60cb

STARK (public x, public y): (0x384cd59e17fcff97e2933bc871202643bf804edb0ac8d6ca2528d08337cd74c,0x5c117d08c6ab261e3de34069f69827b65a829ae540be8225fe64107d053a044)

Account onboarding response:
{'account': {'accountNumber': '0',
             'createdAt': '2022-01-07T02:05:28.222Z',
             'equity': '0.000000',
             'freeCollateral': '0.000000',
             'id': 'd89aeefa-0adc-55d2-96e1-fc5b26826dd5',
             'openPositions': {},
             'pendingDeposits': '0.000000',
             'pendingWithdrawals': '0.000000',
             'positionId': '57275',
             'quoteBalance': '0.000000',
             'starkKey': '0384cd59e17fcff97e2933bc871202643bf804edb0ac8d6ca2528d08337cd74c'},
 'apiKey': {'key': 'd89dca3e-b072-a5df-9d75-3854e723fa84',
            'passphrase': '41Zj4_RqCWnOmtYVQPhz',
            'secret': 'uAgIXkrE0WU